# Create Patches

In [2]:
import os
import numpy as np
from PIL import Image
from itertools import product
import matplotlib.pyplot as plt

In [4]:
for dirname, _, filenames in os.walk('/kaggle/input/fungal-10x'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/fungal-10x/N012a013.tif
/kaggle/input/fungal-10x/F052a17.tif
/kaggle/input/fungal-10x/F021a01.tif
/kaggle/input/fungal-10x/N004a019.tif
/kaggle/input/fungal-10x/F033a12.tif
/kaggle/input/fungal-10x/F017a09.tif
/kaggle/input/fungal-10x/F052a06.tif
/kaggle/input/fungal-10x/N005a001.tif
/kaggle/input/fungal-10x/N004a010.tif
/kaggle/input/fungal-10x/N004a077.tif
/kaggle/input/fungal-10x/N011a010.tif
/kaggle/input/fungal-10x/F053a01.tif
/kaggle/input/fungal-10x/N005a003.tif
/kaggle/input/fungal-10x/N011a002.tif
/kaggle/input/fungal-10x/F007a08.tif
/kaggle/input/fungal-10x/F058a05.tif
/kaggle/input/fungal-10x/N004a102.tif
/kaggle/input/fungal-10x/N004a027.tif
/kaggle/input/fungal-10x/F018a10.tif
/kaggle/input/fungal-10x/F018a06.tif
/kaggle/input/fungal-10x/N005a020.tif
/kaggle/input/fungal-10x/N004a133.tif
/kaggle/input/fungal-10x/N007a003.tif
/kaggle/input/fungal-10x/F013a11.tif
/kaggle/input/fungal-10x/N005a016.tif
/kaggle/input/fungal-10x/N005a023.tif
/kaggle/input/fungal-10

In [9]:
source_dir = os.path.abspath("/kaggle/input/fungal-10x/")
patch_dir = os.path.abspath('/kaggle/working/patches/')
patch_size = 256

In [11]:
def tile(filename, dir_in, dir_out, d):
    if not os.path.isdir(dir_out):
        os.mkdir(dir_out)

    name, ext = os.path.splitext(filename)
    img = Image.open(os.path.join(dir_in, filename))
    w, h = img.size

    grid = product(range(0, h-h%d, d), range(0, w-w%d, d))
    for i, j in grid:
        box = (j, i, j+d, i+d)
        i /= 256
        j /= 256
        out = os.path.join(dir_out, f'{name}_{int(i)}_{int(j)}{ext}')
        img.crop(box).save(out)


if not os.path.isdir(patch_dir):
    os.mkdir(patch_dir)

for filename in os.listdir(source_dir):
    name, ext = os.path.splitext(filename)
    output_patches_dir = os.path.join(patch_dir, name)
    
    print("Patching", filename)
    tile(filename, source_dir, output_patches_dir, patch_size)

Patching N012a013.tif
Patching F052a17.tif
Patching F021a01.tif
Patching N004a019.tif
Patching F033a12.tif
Patching F017a09.tif
Patching F052a06.tif
Patching N005a001.tif
Patching N004a010.tif
Patching N004a077.tif
Patching N011a010.tif
Patching F053a01.tif
Patching N005a003.tif
Patching N011a002.tif
Patching F007a08.tif
Patching F058a05.tif
Patching N004a102.tif
Patching N004a027.tif
Patching F018a10.tif
Patching F018a06.tif
Patching N005a020.tif
Patching N004a133.tif
Patching N007a003.tif
Patching F013a11.tif
Patching N005a016.tif
Patching N005a023.tif
Patching F030a08.tif
Patching N004a132.tif
Patching N004a005.tif
Patching N004a030.tif
Patching N004a107.tif
Patching N004a062.tif
Patching F056a18.tif
Patching F006a03.tif
Patching N012a020.tif
Patching N004a069.tif
Patching N004a075.tif
Patching N004a094.tif
Patching N004a066.tif
Patching F056a09.tif
Patching N004a084.tif
Patching N011a005.tif
Patching N004a033.tif
Patching F033a29.tif
Patching N004a093.tif
Patching F053a09.tif
Patch

# Feature Extraction

In [2]:
import os
import yaml
import argparse

# import h5py
import cv2
import numpy as np
from PIL import Image

from keras.models import Sequential
from keras.layers import Dense
from keras.utils.np_utils import to_categorical

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input

from tensorflow.keras.utils import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from keras.callbacks import ModelCheckpoint

from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score, confusion_matrix

In [3]:
patch_dir = os.path.abspath('/kaggle/working/patches/')
feat_dir = os.path.abspath('/kaggle/working/features/')

In [6]:
# Create feat_dir if not exists.
if not os.path.isdir(feat_dir):
    os.mkdir(feat_dir)

# Loading ResNet50 wit imagenet weights, include_top means that we loading model without last fully connected layers
model = ResNet50(weights = 'imagenet', include_top = False)

# patch_folders = [os.path.join(patch_dir, folder) for folder in sorted(os.listdir(patch_dir))]
# patches_per_image = len(os.listdir(patch_folders[0]))
# print(patches_per_image)

# Create dataset from the image patches
for folder in sorted(os.listdir(patch_dir)):
    filename = str(folder).split("/")[-1]
    filePath = os.path.join(feat_dir, filename)
    # Run only if file doesn't already exist
    if os.path.exists(filePath):
        print("Skipping File:", filename)
        continue
    print("Running on File:", filename)

    features = []
    patch_folder = os.path.join(patch_dir, folder)
    for patch_file in sorted(os.listdir(patch_folder)):
        img_path = os.path.join(patch_folder, patch_file)

        # Get coord in [x, y] format
        coord = img_path.split("/")
        coord = coord[-1]
        coord = coord.split(".")[-2]
        coord = coord.split("_")
        coord = [int(coord[-2])/256, int(coord[-1])/256]

        # Read image
        orig = cv2.imread(img_path)

        # Convert image to RGB from BGR (another way is to use "image = image[:, :, ::-1]" code)
        orig = cv2.cvtColor(orig, cv2.COLOR_BGR2RGB)

        # Resize image to 224x224 size
        image = cv2.resize(orig, (224, 224)).reshape(-1, 224, 224, 3)

        # We need to preprocess imageto fulfill ResNet50 requirements
        image = preprocess_input(image)

        # Extracting our features
        feature = model.predict(image)

        # Group the features
        features.append(feature)
    np.save(filePath, features)

Running on File: F005a02
Running on File: F006a01
Running on File: F006a02
Running on File: F006a03
Running on File: F006a04
Running on File: F006a05
Running on File: F006a06
Running on File: F006a07
Running on File: F006a08
Running on File: F006a09
Running on File: F006a10
Running on File: F007a01
Running on File: F007a02
Running on File: F007a03
Running on File: F007a04
Running on File: F007a05
Running on File: F007a06
Running on File: F007a07
Running on File: F007a08
Running on File: F007a09
Running on File: F007a10
Running on File: F007a11
Running on File: F007a12
Running on File: F007a13
Running on File: F007a15
Running on File: F007a16
Running on File: F007a17
Running on File: F007a18
Running on File: F007a19
Running on File: F007a21
Running on File: F007a22
Running on File: F009a01
Running on File: F009a02
Running on File: F009a03
Running on File: F009a04
Running on File: F010a01
Running on File: F010a02
Running on File: F010a03
Running on File: F010a04
Running on File: F010a05


# Dataset creator

In [8]:
filename = os.path.join('/kaggle/working/', 'fungal_vs_nonfungal.csv')
patch_dir = os.path.abspath('/kaggle/working/patches/')
feat_dir = os.path.abspath('/kaggle/working/features/')
annotated_dir = os.path.abspath('/kaggle/input/fungal-10x-annot/')

In [9]:
with open(filename, 'w') as file:
    file.write('case_id,slide_id,label' + '\n')

    patch_folders = [os.path.join(patch_dir, folder) for folder in sorted(os.listdir(patch_dir))]

    for i, name in enumerate(patch_folders):
        name = name.split("/")[-1]
        if name != feat_dir:
            if name[0] == "F":
                f_nf = "fungal"
            elif name[0] == "N":
                f_nf = "nonfungal"
                annotated = True
            else:
                f_nf = "unclassified"

            line = 'case_' + str(i) + ',' + name + ',' + f_nf
            file.write('{}\n'.format(line))


# Create Splits